In [1]:
import pandas as pd
import requests

In [2]:
symbol = "EMAAR"
exchange = "DFM"  # ADSM or DFM, ADSM = ADX
session_id = "e23b8b7e-5e52-4ce8-ade6-0c4e3311b720"
start = "1900-01-31T20:18:48.000Z"
end = "2100-01-31T20:00:00.000Z"
interval = "1"  # 1 days
additional_slug = "1753968492999"

url = f"https://mobile.intlsecurities.ae/FITDataFeedServiceGateway/DataFeedService.asmx/datafeedDFN?session={session_id}&symbol={symbol},{exchange}&period=day&from={start}&to={end}&interval={1}${additional_slug}"

In [3]:
def fetch_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()

        data = response.json()
        df = pd.DataFrame(data)

        if "DT" in df.columns:
            df.rename(columns={"DT": "Date"}, inplace=True)
            df["Date"] = pd.to_datetime(df["Date"]).dt.date
            df = df.set_index("Date")

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
    except ValueError:
        print("Error: Could not decode JSON from the response.")

    return df, data


df, data = fetch_data(url)
print(data[0:5])
print(df.head())

[{'DT': '2025-08-07T00:00:00.000Z', 'Open': 15.3, 'High': 15.5, 'Low': 15.25, 'Close': 15.5, 'Volume': 2983424}, {'DT': '2025-08-06T00:00:00.000Z', 'Open': 15.5, 'High': 15.5, 'Low': 15.35, 'Close': 15.45, 'Volume': 11745540}, {'DT': '2025-08-05T00:00:00.000Z', 'Open': 15.3, 'High': 15.55, 'Low': 15.25, 'Close': 15.45, 'Volume': 6482941}, {'DT': '2025-08-04T00:00:00.000Z', 'Open': 15.35, 'High': 15.35, 'Low': 15.05, 'Close': 15.35, 'Volume': 8367905}, {'DT': '2025-08-01T00:00:00.000Z', 'Open': 15.25, 'High': 15.4, 'Low': 15.1, 'Close': 15.35, 'Volume': 7201568}]
             Open   High    Low  Close    Volume
Date                                            
2025-08-07  15.30  15.50  15.25  15.50   2983424
2025-08-06  15.50  15.50  15.35  15.45  11745540
2025-08-05  15.30  15.55  15.25  15.45   6482941
2025-08-04  15.35  15.35  15.05  15.35   8367905
2025-08-01  15.25  15.40  15.10  15.35   7201568


In [4]:
trans_log = pd.read_csv("raw-data/IntlSecurities_July2025.csv")
symbol_market_pair = trans_log[["Symbol", "Exchange"]].dropna().drop_duplicates()
symbol_market_pair

,Symbol,Exchange
1,ALDAR,ADX
2,EMAAR,DFM
3,CHADX15,ADX
4,BURJEEL,ADX
5,MULTIPLY,ADX
6,IHC,ADX
8,AMR,ADX
9,ASM,ADX


In [ ]:
for _, row in symbol_market_pair.iterrows():
    symbol = row["Symbol"]
    exchange = row["Exchange"] if row["Exchange"] == "DFM" else "ADSM"
    url = f"https://mobile.intlsecurities.ae/FITDataFeedServiceGateway/DataFeedService.asmx/datafeedDFN?session={session_id}&symbol={symbol},{exchange}&period=day&from={start}&to={end}&interval={1}${additional_slug}"
    df, _ = fetch_data(url)
    df.reset_index().to_csv(f"../data/manual-source/prices/{symbol}.csv", index=False)

![test.png](IntlSecurities-data-scrapping.png)